In [53]:
using CSV
using DataFrames
using Printf
using DelimitedFiles
import PyPlot; const plt = PyPlot # now do plt.plt[:hist] for example
using MultivariateStats
using Optim

In [54]:
list_of_isotherms = readdlm("../data/expt_data/list_of_isotherms.txt")

10×1 Array{Any,2}:
 "Al-MIL-53_304K.csv"  
 "MIL-101_303K.csv"    
 "MIL-101-NH2_293K.csv"
 "MIL-101-NO2_293K.csv"
 "MIL-125_298K.csv"    
 "MIL-125-NH2_298K.csv"
 "MIL-68_298K.csv"     
 "MIL-68-NH2_298K.csv" 
 "SIM-1_303K.csv"      
 "ZIF-8_298K.csv"      

In [55]:
function read_and_convert(isotherm_file::AbstractString)
    @printf("Working on file %s\n", isotherm_file)
    isotherm = CSV.read("../data/expt_data/" * isotherm_file)
    pUnit, aUnit = string.(names(isotherm))
    
    if pUnit == "Pressure(bar)"
        @printf("Pressure unit is already in bar\n")
    elseif pUnit == "Pressure(torr)"
        isotherm[1] = isotherm[1] ./ 750.06
        @printf("Converting pressure unit from torr to bar\n")
    elseif pUnit == "Pressure(kPa)"
        isotherm[1] = isotherm[1] ./ 100.0
        @printf("Converting pressure unit from kPa to bar\n")
    else
        @printf("WHAT? P\n")
    end
    names!(isotherm, [Symbol("Pressure(bar)"), names(isotherm)[2]])
            
    MW_CO2 = 44.01
    if aUnit == "Adsorption(mmol/g)"
        @printf("Adsorption unit is already in mmol/g\n")
    elseif aUnit == "Adsorption(mg/g)"
        isotherm[2] = isotherm[2] ./ MW_CO2
        @printf("Converting adsorption unit from mg/g to mmol/g\n")
    elseif aUnit == "Adsorption(cm3(STP)/g)"
        isotherm[2] = isotherm[2] ./ 22.4
        @printf("Converting adsorption unit from cm3(STP)/g to mmol/g\n")
    else
        @printf("WHAT? A\n")
    end
    names!(isotherm, [names(isotherm)[1], Symbol("Adsorption(mmol/g)")])
    return isotherm
end

read_and_convert (generic function with 1 method)

In [56]:
function fit_henry(df_ads::DataFrame, incl_nb_pts::Int)
    # sort by pressure
    sort!(df_ads, [Symbol("Pressure(bar)")])
    # build feature matrix
    P = zeros(incl_nb_pts, 1)
    n = zeros(incl_nb_pts)
    for (i, row) in enumerate(eachrow(df_ads))
        if i > incl_nb_pts
            break
        end
        P[i, 1] = row[Symbol("Pressure(bar)")]
        n[i] = row[Symbol("Adsorption(mmol/g)")]
    end
    # fit line without bias (henry coef) to determine henry coefficient
    K = llsq(P, n; bias=false)[1]
    return K # mmol/(g-bar)
end

fit_henry (generic function with 1 method)

In [59]:
nb_pts_to_incl_in_fitting = Dict{AbstractString, Int64}()
nb_pts_to_incl_in_fitting["Al-MIL-53_304K.csv"] = 2
nb_pts_to_incl_in_fitting["MIL-101_303K.csv"] = 3
nb_pts_to_incl_in_fitting["MIL-101-NH2_293K.csv"] = 5
nb_pts_to_incl_in_fitting["MIL-101-NO2_293K.csv"] = 6
nb_pts_to_incl_in_fitting["MIL-125_298K.csv"] = 10
nb_pts_to_incl_in_fitting["MIL-125-NH2_298K.csv"] = 10
nb_pts_to_incl_in_fitting["MIL-68_298K.csv"] = 5
nb_pts_to_incl_in_fitting["MIL-68-NH2_298K.csv"] = 4
nb_pts_to_incl_in_fitting["SIM-1_303K.csv"] = 3
nb_pts_to_incl_in_fitting["ZIF-8_298K.csv"] = 3

3

In [74]:
open("co2_henry_coefs.csv", "w") do f
    @printf(f, "MOF,CO2_henry coefficient [mmol/(g-bar)],Temperature [K]\n")
    for MOF in list_of_isotherms
        isotherm = read_and_convert(MOF)
        henry = fit_henry(isotherm, nb_pts_to_incl_in_fitting[MOF])
        mofstuff = split(MOF, "_")
        println(mofstuff[2])
        temperature = split(mofstuff[2], "K.")[1]
        mofname = mofstuff[1]
        @printf(f, "%s,%f,%d\n", mofname, henry, parse(Int64, temperature))
    end
end

Working on file Al-MIL-53_304K.csv
Pressure unit is already in bar
Adsorption unit is already in mmol/g
304K.csv
Working on file MIL-101_303K.csv
Pressure unit is already in bar
Adsorption unit is already in mmol/g
303K.csv
Working on file MIL-101-NH2_293K.csv
Converting pressure unit from torr to bar
Adsorption unit is already in mmol/g
293K.csv
Working on file MIL-101-NO2_293K.csv
Converting pressure unit from torr to bar
Adsorption unit is already in mmol/g
293K.csv
Working on file MIL-125_298K.csv
Converting pressure unit from kPa to bar
Converting adsorption unit from mg/g to mmol/g
298K.csv
Working on file MIL-125-NH2_298K.csv
Converting pressure unit from kPa to bar
Converting adsorption unit from mg/g to mmol/g
298K.csv
Working on file MIL-68_298K.csv
Converting pressure unit from torr to bar
Adsorption unit is already in mmol/g
298K.csv
Working on file MIL-68-NH2_298K.csv
Converting pressure unit from torr to bar
Adsorption unit is already in mmol/g
298K.csv
Working on file SI